# 2015 Flight Delays and Cancellations

## Imports

Preferably, place any additional imports you will need throughout the notebook in the cell below.

In [2]:
import pandas as pd
from numpy import int64, float64

## Extract and Load Data

In [3]:
AIRLINES_CSV_PATH = 'airlines.csv'
AIRPORTS_CSV_PATH = 'airports.csv'
FLIGHTS_CSV_PATH = 'flights.csv'

### TODO 1

Iterate over the csv_paths, table_names, and index_cols simultaneously using zip. This will give you access to each CSV path, its corresponding table name, and the index column in each iteration. 

For each iteration, implement the code where the data is loaded into a pandas DataFrame from the provided csv path (`pd.read_csv` function) using the respective value of index_col for the `index_col` parameter of the `pd.read_csv`. Store the dataframe as a value in the `df_dict` under the `table_name` as key.

In [4]:
def generate_pandas_df_dict(csv_paths, table_names=None, index_cols=None):
    if table_names is None:
        table_names = csv_paths
    if index_cols is None:
        index_cols = (None,) * len(csv_paths)
    df_dict = {}
    for i, path in enumerate(csv_paths):
        df_dict[table_names[i]] = pd.read_csv(path, index_col=index_cols[i])
    return df_dict

"""
df_dict = generate_pandas_df_dict(
    (AIRLINES_CSV_PATH, AIRPORTS_CSV_PATH, FLIGHTS_CSV_PATH),
    ('airlines', 'airports', 'flights'),
    (0, 0, None)
)

df_dict
"""

/tmp/ipykernel_27909/3558430354.py:8: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[table_names[i]] = pd.read_csv(path, index_col=index_cols[i])


{'airlines':                                 AIRLINE
 IATA_CODE                              
 UA                United Air Lines Inc.
 AA               American Airlines Inc.
 US                      US Airways Inc.
 F9               Frontier Airlines Inc.
 B6                      JetBlue Airways
 OO                Skywest Airlines Inc.
 AS                 Alaska Airlines Inc.
 NK                     Spirit Air Lines
 WN               Southwest Airlines Co.
 DL                 Delta Air Lines Inc.
 EV          Atlantic Southeast Airlines
 HA               Hawaiian Airlines Inc.
 MQ         American Eagle Airlines Inc.
 VX                       Virgin America,
 'airports':                                        AIRPORT  \
 IATA_CODE                                        
 ABE        Lehigh Valley International Airport   
 ABI                   Abilene Regional Airport   
 ABQ          Albuquerque International Sunport   
 ABR                  Aberdeen Regional Airport   
 ABY         

## Show Tables

### TODO 2

Implement the code so that the function prints output of the following shape:

    === TABLES ===
    - airlines (14 rows, 1 columns)
    - airports (322 rows, 6 columns)
    - flights (5819079 rows, 31 columns)
    TOTAL 3

Hint: Use the 'shape' attribute of each Dataframe to get the rows and columns.    

In [5]:
def show_tables(df_dict):
    print('=== TABLES ===')
    for name, df in sorted(df_dict.items()):
        rows, cols = df.shape
        print(f'- {name} ({rows} rows, {cols} columns)')
    print(f'TOTAL {len(df_dict)}')

# show_tables(df_dict)

=== TABLES ===
- airlines (14 rows, 1 columns)
- airports (322 rows, 6 columns)
- flights (5819079 rows, 31 columns)
TOTAL 3


## Describe Table

### TODO 3
Complete the code in the `if` branch with max, min and mean functions of the Dataframe's column i.e., replace
<MAX_FUNCTION>,<MIN_FUNCTION> and <MEAN_FUNCTION> with appropriate functions. 

Finish the `else` branch so that a column that is not int64 or float64 will have the following shape:

    - TAIL_NUMBER object (5804358/5819079) [4898 unique]

Note that for the count of unique objects an absenting value counts as one. For example, if there is a column where lines are either assigned with value "red" or value "blue", and there is a couple of line where the value is not assigned, then the column should be reported as having three unique values.

In [6]:
def describe_table(df):
    print('=== COLUMNS ===')
    total = df.shape[0]
    for column, dtype in zip(df.columns, df.dtypes):
        series = df[column]
        f_string = f'- {column} {dtype} ({series.count()}/{total}) '
        if dtype == int64 or dtype == float64:
            print(f_string + f'[Max: {series.max()}, Mean: {series.mean()}, Min: {series.min()}]' )
        else:
            if column != 'CANCELLATION_REASON':
                print(f_string + f'[{series.nunique()} unique]')
            else:
                print(f_string + f'[{len(list(df["CANCELLATION_REASON"].unique()))} unique]')
    print(f'TOTAL {total}')

# describe_table(df_dict['airports'])

=== COLUMNS ===
- AIRPORT object (322/322) [322 unique]
- CITY object (322/322) [308 unique]
- STATE object (322/322) [54 unique]
- COUNTRY object (322/322) [1 unique]
- LATITUDE float64 (319/322) [Max: 71.28545, Mean: 38.9812439184953, Min: 13.48345]
- LONGITUDE float64 (319/322) [Max: -64.79856, Mean: -98.37896445141067, Min: -176.64603]
TOTAL 322


## Identify Useless Columns

### TODO 4

Add one additional check that will assess if the series has only a single value. If yes the following should be printed (COLUMN_NAME will replaced by the actual name of the column):

    The column COLUMN_NAME appears to have only one value.

Hint: Think something on the lines of a method that gives unique values having a parameter that cleans NaN data.

In [7]:
def assess_useless_columns(df):
    for column in df.columns:
        series = df[column]
        if series.count() == 0:
            print(f'The column {column} appears to have no values.')
        if len(series.unique()) == 1:
            print(f'The column {column} appears to have only one value.')

# assess_useless_columns(df_dict['airports'])

The column COUNTRY appears to have only one value.


## Delays Per Airline

The function `generate_delays_per_airline_csv` produces the following csv stored at the `csv_path`:

    AIRLINE,ARRIVAL_DELAY
    Spirit Air Lines,14.471799501705833
    Frontier Airlines Inc.,12.504706404706404
    JetBlue Airways,6.677860800940307
    Atlantic Southeast Airlines,6.585378691739733
    American Eagle Airlines Inc.,6.457873460764516
    Skywest Airlines Inc.,5.845652151300072
    United Air Lines Inc.,5.431593935741549
    Virgin America,4.737705721003135
    Southwest Airlines Co.,4.3749636792570525
    US Airways Inc.,3.7062088424131026
    American Airlines Inc.,3.4513721447256764
    Hawaiian Airlines Inc.,2.023092805197196
    Delta Air Lines Inc.,0.18675361236390797
    Alaska Airlines Inc.,-0.9765630924118783

Uncomment the commented lines below and fill the values for <COLUMN_1> and <COLUMN_2>. Look at the output above, observe the column names and use them to complete the code. 

In [8]:
def generate_delays_per_airline_csv(df_dict, csv_path):
    df_dict_delay = df_dict['flights'][['AIRLINE','ARRIVAL_DELAY']]
    mean = df_dict_delay.groupby('AIRLINE').mean()
    pd.merge(df_dict['airlines'],
        mean,
        left_index=True,
        right_index=True,
    ).sort_values(
        by='ARRIVAL_DELAY',
        ascending=False,
        ignore_index=True
    ).to_csv(csv_path, index=False)

# generate_delays_per_airline_csv(df_dict, 'out.csv')

## Delays Per Arrival Airport

### TODO 5

Implement the `generate_delays_per_arriving_airport_csv` function that works similarly to the `generate_delays_per_airline_csv`. However, instead of listing the average dalays per airline list them per airport where the flight arrives. The output should contain the name of the arriving airport in the first column and the average delay in the second.

In [9]:
def generate_delays_per_arriving_airport_csv(df_dict, csv_path):
    df_dict_delay = df_dict['flights'][['DESTINATION_AIRPORT','ARRIVAL_DELAY']]
    pd.merge(df_dict_delay.groupby('DESTINATION_AIRPORT').mean(),
        df_dict['airports'].loc[:, ['AIRPORT']],
        left_on='DESTINATION_AIRPORT',
        right_on='IATA_CODE',
    ).sort_values(
        by='ARRIVAL_DELAY', ascending=False, ignore_index=True
    )[['AIRPORT', 'ARRIVAL_DELAY']].to_csv(csv_path, index=False)

# generate_delays_per_arriving_airport_csv(df_dict, 'out.csv')